In [ ]:
# other stuff
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', 50)
from IPython.display import display, HTML

# """Minimal implementation of Wasserstein GAN for MNIST."""
from tensorflow.contrib import layers

import tensorflow as tf
import numpy as np
import os
import math
import itertools
from utils import *

# Save
import arff
def save_arff(df,filename):
    filename=filename.replace(".csv",".arff")
    arff.dump(filename, df.values, relation='Relation', names=df.columns)
    if len(df.Class.unique()) == 1: 
        print("=1 class:",df.Class.unique())
    else:
        print(">1 class:",df.Class.unique())
        
    _max = 10 if "MNIST" in filename else 2
    classes = "{" + ",".join([str(x) for x in range(_max)]) + "}"
    !sed -i "s/Class string/Class {classes}/g" {filename}
    
def save_data(data=None, save_file=None, attribute_names=None, num_classes=1):
    #invalid attribute names for weka
    if data.shape[1] ==36:attribute_names=["attr_"+str(i+1) for i in range (len(attribute_names))]
    attributes = data[:, :-num_classes]
    classes    = data[:, -num_classes:]

    if num_classes > 1:
        # MNIST class is the index of largest row.
        _class = np.argmax(classes,axis=1)
    else:
        # Other class are defined by the conditions x>=.5 -> 1 and x<.5 -> 0
        _class = np.around(classes)
    print(save_file,"mean:",np.mean(_class),"| std:",np.std(_class),end=" | ")
    # Last column must be Class as int.
    df=pd.DataFrame(attributes,columns=attribute_names)
#     print(_class)
    df["Class"]=_class.astype(int).astype(str)
    df.to_csv(save_file,index=False)
    save_arff(df, save_file)
    
    # If saving the MNIST_data also save the expanded class for training on the wgan.
    if num_classes > 1:
        df=pd.DataFrame(attributes,columns=attribute_names)
        for i in range(num_classes):
            df["Class_{}".format(i)]=classes[:,i]
        df.to_csv(save_file.replace(".csv","_for_DPWGAN.csv"),index=False)


class Data_Loader:
    def __init__(self,data):
        self.data = data
        self.data_len = data.shape[0]
        self.i = -1
    def next_batch(self, batch_size):
        max_i=int(len(self.data)/batch_size)
        self.i += 1
        if self.i == max_i: self.i = 0
        return self.data[batch_size*self.i: (self.i+1)*batch_size]

# Use this if you don't want to preprocess data again.
def Load_data():
    # Originally data loaded into global varibles during preproccessing.
    global attribute_names, all_data
    train_df_kccfd=pd.read_csv("DATA/PREPROCESSED/TRAIN/KCCFD_TRAIN.csv")
    train_df_kccr=pd.read_csv("DATA/PREPROCESSED/TRAIN/KCCR_TRAIN.csv")
    train_df_mnist=pd.read_csv("DATA/PREPROCESSED/TRAIN/MNIST_TRAIN_for_DPWGAN.csv")
    attribute_names={}
    attribute_names["KCCR"]=list(train_df_kccr)[:-1]
    attribute_names["KCCFD"]=list(train_df_kccfd)[:-1]
    attribute_names["MNIST"]=list(train_df_mnist)[:-10]
    train_mnist=train_df_mnist.values
    train_kccfd=train_df_kccfd.values
    train_kccr=train_df_kccr.values
    all_data={"MNIST": Data_Loader(train_mnist),
          "KCCFD": Data_Loader(train_kccfd),
          "KCCR":  Data_Loader(train_kccr)}  
Load_data()

In [ ]:
#network archetecture
def get_nets(N):
    
    def leaky_relu(x): return tf.maximum(x, 0.2 * x)
    
    if N=="MNIST":
        # Weights based on the implementation here:
        # https://github.com/Zackory/Keras-MNIST-GAN/blob/master/mnist_gan.py
        def generator(z):
            with tf.variable_scope('generator'):
                z = layers.fully_connected(z, num_outputs=256, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=512, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=1024, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=794, activation_fn=leaky_relu)
                return z
            
        def discriminator(x, reuse, c_p=.01):
            with tf.variable_scope('discriminator', reuse=reuse, constraint=lambda x: tf.clip_by_value(x,-c_p,c_p)):
                x = layers.fully_connected(x, num_outputs=1024, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=512, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=256, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=1, activation_fn=leaky_relu)
                return x
            
    elif N=="KCCFD":
        def generator(z):
            with tf.variable_scope('generator'):
                z = layers.fully_connected(z, num_outputs=512, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=228, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=30, activation_fn=tf.nn.sigmoid)
                return z
            
        def discriminator(x, reuse, c_p=.01):
            with tf.variable_scope('discriminator', reuse=reuse, constraint=lambda x: tf.clip_by_value(x,-c_p,c_p)):
                x = layers.fully_connected(x, num_outputs=1024, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=1024, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=512, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=1, activation_fn=leaky_relu)
                return x
            
    elif N=="KCCR":
        def generator(z):
            with tf.variable_scope('generator'):
                z = layers.fully_connected(z, num_outputs=128, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=128, activation_fn=leaky_relu)
                z = layers.fully_connected(z, num_outputs=36, activation_fn=tf.nn.sigmoid)
                return z
            
        def discriminator(x, reuse, c_p=.01):
            with tf.variable_scope('discriminator', reuse=reuse, constraint=lambda x: tf.clip_by_value(x,-c_p,c_p)):
                x = layers.fully_connected(x, num_outputs=128, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=128, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=32, activation_fn=leaky_relu)
                x = layers.fully_connected(x, num_outputs=1, activation_fn=leaky_relu)
                return x
            
    return generator, discriminator


class TF_model:
    def __init__(self, name=None,T="WGAN",dataset="MNIST",batch_size=50,epsilon=1.0,overide_std=None):
        assert T in ["WGAN","DPWGAN"]
        assert dataset in ["MNIST","KCCFD","KCCR"]
        self.data=all_data[dataset]

        self.name = ("_".join([x for x in [T,dataset,name] if x]).strip())
        self.dataset = dataset
        self.type = T
        self.ckpt_name = "models/"+self.name+"/model.ckpt"
        self.batch_size = batch_size
        self.epsilon = epsilon
        self.step = 0 # Track number of steps for saving.
        self.n_d = 5 # Discriminator_iterations_per_step = 5
        self.c_p = 0.01 # Clipping perameter
        
        # For DPWGAN, calcualted std for differential privacy.
        if self.type == "DPWGAN": 
            if overide_std: self.std=overide_std
            else: self.std = self.calculate_std(epsilon) 
            
        self.graph = tf.Graph()
        self.sess = tf.InteractiveSession(graph=self.graph)
        self.make_model()
        with self.graph.as_default():
            self.saver = tf.train.Saver(max_to_keep=6)
            
            
    # calculate std for noise to ensure differential privacy
    def calculate_std(self,epsilon):
        delta=10**(-8)
        m=self.batch_size # Batch size
        M=self.data.data_len   # Data  size
        q=m/M # Sampling Probability

        B_sigma = 1 # Upper bound on the activation function
        B_sigma_prime = 1/4  # Upper bound on the derivitive activation function

        # Number of connections
        # Cheese this number by creating another model.
        num_gradients = TF_model(dataset=self.dataset,T="WGAN",batch_size=1).calc_num_grads()
        print("self.c_p:",self.c_p,"B_sigma:",B_sigma,"B_sigma_prime:",B_sigma_prime,"num_gradients:",num_gradients)
        bound_of_dW_distance= 2*(self.c_p)*(B_sigma)*(B_sigma_prime**2)*num_gradients
        print("bound_of_dW_distance",bound_of_dW_distance)
        print("q:",q,"self.n_d:",self.n_d,"delta:",delta,"epsilon:",self.epsilon)
        noise_scale=2*q*(self.n_d*math.sqrt(math.log(1/delta) / self.epsilon))
        std = noise_scale*bound_of_dW_distance
        print("std =",std)
        return std
    
    
    def make_model(self):
        global LR
        with self.sess.as_default():

            # Define noise layer
            def get_DP_gradients(gradients):
                noise = tf.random_normal(shape=tf.shape(gradients), mean=0.0,
                                         stddev=self.std, dtype=tf.float32)
                return gradients + noise

            # Set placeholders
            with tf.name_scope('placeholders'):
                def get_placeholder(N):
                    # input placeholder, this must be constucted withing the varible scope.
                    
                    if   N=="MNIST": return tf.placeholder(tf.float32, [None,  794])
                    elif N=="KCCFD": return tf.placeholder(tf.float32, [None,  30])
                    elif N=="KCCR":  return tf.placeholder(tf.float32, [None,  36])
                self.z = tf.placeholder(tf.float32, [None, 128])
                self.x_true = get_placeholder(self.dataset)

            # Initialize nets
            generator, discriminator = get_nets(self.dataset)
            self.x_generated = generator(self.z)
            self.d_true = discriminator(self.x_true, reuse=False, c_p=self.c_p)
            self.d_generated = discriminator(self.x_generated, reuse=True, c_p=self.c_p)

            with tf.name_scope('regularizer'):
                self.epsilon = tf.random_uniform([self.batch_size, 1, 1, 1], 0.0, 1.0)
                self.x_hat = self.epsilon * self.x_true + (1 - self.epsilon) * self.x_generated
                self.d_hat = discriminator(self.x_hat, reuse=True)
                self.gradients = tf.gradients(self.d_hat, self.x_hat, name="gradients")[0]
                if self.type == "WGAN":
                    self.ddx = tf.sqrt(tf.reduce_sum(self.gradients ** 2, axis=[1, 2]))
                elif self.type == "DPWGAN": # differential private gradients
                    self.gradients = get_DP_gradients(self.gradients)
                    self.ddx = tf.sqrt(tf.reduce_sum(self.gradients ** 2, axis=[1, 2]))
                self.d_regularizer = tf.reduce_mean((self.ddx - 1.0) ** 2)

            with tf.name_scope('loss'):
                self.g_loss = tf.reduce_mean(self.d_generated)
                self.d_loss = (tf.reduce_mean(self.d_true) - tf.reduce_mean(self.d_generated) +
                          10 * self.d_regularizer)

            with tf.name_scope('optimizer'):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=LR, beta1=0, beta2=0.9)
                self.g_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator')
                self.g_train = self.optimizer.minimize(self.g_loss, var_list=self.g_vars)
                self.d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator')
                self.d_train = self.optimizer.minimize(self.d_loss, var_list=self.d_vars)
                
            tf.global_variables_initializer().run()
            
            
    def train(self,n=20000,save_every=20):
        with self.graph.as_default():
            for i in range(1,n+1):
                # Inputs
                batch = self.data.next_batch(self.batch_size)
                z_train = np.random.randn(self.batch_size, 128)
                # g-train
                g_loss, _ =self.sess.run([self.g_loss,self.g_train], feed_dict={self.z: z_train})
                # d-train
                for j in range(self.n_d): d_loss,_=self.sess.run([self.d_loss,self.d_train],  feed_dict={self.x_true: batch, self.z: z_train})
                # Bookkeeping
                self.step+=1
                if i % 20 == 0 or i==n: print("d_loss:",d_loss,"| g_loss:",g_loss,end=" |")
                if (save_every and i % 20 == 0) or i==n: self.save_samples()
                if i % 100 == 0 or i==n:
                    print('train iter={}/{}, step={}'.format(i,n,self.step))
                    self.generate(1)
        return self
    
    
    def calc_num_grads(self):
        with self.graph.as_default():
            from functools import reduce
            def mult(shape): return reduce(lambda x, y: x*y, shape)
            # Inputs
            batch = self.data.next_batch(self.batch_size)
            z_train = np.random.randn(self.batch_size, 128)
            # Get a set of gradients    
            grads=self.sess.run(self.gradients, feed_dict={self.x_true: batch, self.z: z_train})
            # Count them up
            total_gradients = 0
            for gradient in grads:
                print(gradient.shape)
                variable_gradients=mult(gradient.shape)
                print("max grad:",np.amax(gradient),"| min grad:",np.amin(gradient))
                print(variable_gradients)
                total_gradients += variable_gradients
            print("total_gradients =",total_gradients/self.batch_size)
            return total_gradients
        
        
    def generate(self,n=5):
        with self.graph.as_default():
            if self.dataset == "MNIST":
                z_validate = np.random.randn(n, 128)
                generated = self.x_generated.eval(feed_dict = {self.z: z_validate}).squeeze()
                imgs,lables =  np.reshape(generated[:28*28],(-1,28,28)),generated[28*28:]
                self._plot(imgs,lables)
                if len(lables.shape) == 1: self._plot(imgs,lables)
                else:
                    for i in len(lables): self._plot(imgs[i],lables[i])
        return self
    
    
    def save_samples(self,batch_size=50,batches=120):
        with self.graph.as_default():
            samples=[]
            for i in range(batches):
                z_validate = np.random.randn(batch_size, 128)
                generated = self.x_generated.eval(feed_dict = {self.z: z_validate}).squeeze()
                samples+=[x.tolist() for x in generated]
        samples=np.asarray(samples)
        num_classes = 1
        if self.dataset == "MNIST": num_classes = 10
        save_file="DATA/GENERATED/"+self.name+"_GENERATED.csv"
        save_data(samples, save_file, attribute_names[self.dataset], num_classes)
        return self
    
    
    def save_200_samples(self,batch_size=50,batches=40):
        with self.graph.as_default():
            assert batch_size * batches==2000
            samples=[]
            for i in range(batches):
                z_validate = np.random.randn(batch_size, 128)
                generated = self.x_generated.eval(feed_dict = {self.z: z_validate}).squeeze()
                samples+=[x.tolist() for x in generated]
        samples=np.asarray(samples)
        num_classes = 1
        if self.dataset == "MNIST": num_classes = 10
        save_file="DATA/GENERATED/"+self.name+"_GENERATED.csv"
        save_data(samples, save_file, attribute_names[self.dataset], num_classes)
        print("Samples saved:",save_file)
        return self
    
    
    def _plot(self, imgs,lables):
        if len(lables.shape) == 1: i_max=1;imgs=[imgs],lables=[lables]
        else: i_max=lables.shape[0]
        fig, ax = plt.subplots(1, i_max, figsize=(12, 6))
        for i in range(i_max):
            lable=lables[i]
            print("Class =",np.argmax(lable),"class array:",[round(x,2) for x in lable])
            ax[i].set_title("class="+str(np.argmax(lable)))
            ax[i].imshow(np.reshape(img,(28,28)), clim=[0, 1], cmap='bone')
        plt.tight_layout()
        plt.show()
        
        
    def save(self):
        with self.graph.as_default():
            self.saver.save(self.sess, self.ckpt_name, global_step=self.step)
        return self
    
    
    def load(self,path=None):
        #this probably doesn't work
        with self.graph.as_default():
            if path:
                print(tf.train.latest_checkpoint(self.name+"/checkpoint"))
                print(os.path.join(self.name, path))
                self.saver.restore(self.sess, os.path.join(self.name, path))
                print("Loaded sucessfully")
            else:
                print(self.name)
                ckpt = tf.train.get_checkpoint_state(self.name)
                print(os.path.basename(ckpt.model_checkpoint_path))
                if ckpt and ckpt.model_checkpoint_path:
                    print(ckpt.model_checkpoint_path)
                    self.saver.restore(self.sess, os.path.join(path, os.path.basename(ckpt.model_checkpoint_path)))
                    print("Loaded sucessfully")
                else:
                    print("Load error: No checkpoint at "+self.name)
        return self

In [ ]:
def train_WGAN(name=None,batch_size=20,iterations=2000):
    global LR
    LR=2*1e-4
    model_TEST1=TF_model(T="WGAN", name=name ,dataset="KCCR", batch_size=batch_size).train(iterations).save().save_samples()
    LR=5*1e-5
    model_TEST2=TF_model(T="WGAN", name=name ,dataset="KCCFD",batch_size=batch_size).train(iterations).save().save_samples()
    model_TEST3=TF_model(T="WGAN", name=name ,dataset="MNIST",batch_size=batch_size).train(iterations,None).save().save_samples()
# train_WGAN(name="my_data")

def train_DPWGAN(name=None,batch_size=20,epsilon=1,iterations=2000):
    global LR
    LR=2*1e-4
    model_TEST4=TF_model(T="DPWGAN",dataset="KCCR", name=name,batch_size=batch_size,
                 epsilon=epsilon).train(iterations,100).save().save_samples()
    LR=5*1e-5
    model_TEST5=TF_model(T="DPWGAN",dataset="KCCFD",name=name,batch_size=batch_size,
                 epsilon=epsilon).train(iterations,100).save().save_samples()
    model_TEST6=TF_model(T="DPWGAN",dataset="MNIST",name=name,batch_size=batch_size,
                 epsilon=epsilon).train(iterations,100).save().save_samples()
# train_DPWGAN(name="my_data")

def train_all_DPWGANs(batch_size=20,iterations=2000):
    global LR
    for epsilon in [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0][::-1]:
        train_DPWGAN(name="eps="+str(epsilon),batch_size=batch_size,epsilon=epsilon,iterations=iterations)
# train_all_DPWGANs(name="my_data")
